In [36]:
import pymysql
import streamlit as st

@st.cache_resource
def get_tidb_connection():
    return pymysql.connect(
        host=st.secrets["tidb"]["host"],
        port=st.secrets["tidb"]["port"],
        user=st.secrets["tidb"]["user"],
        password=st.secrets["tidb"]["password"],
        ssl_ca=st.secrets["tidb"]["ssl_ca"],
        ssl_verify_cert=True
    )

mydb = get_tidb_connection()
mycursor = mydb.cursor()


In [12]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS IMDB")

0

In [30]:
import pandas as pd

df = pd.read_csv("imdb_movie_list_Cleaned.csv")

In [31]:
df = df[df['Duration'] <= 250]

In [ ]:
mycursor.execute("USE IMDB")

mycursor.execute("""CREATE TABLE IF NOT EXISTS imdb_movies_list(
                 ID int auto_increment primary key,
                 Title varchar(255),
                 Ratings float,
                 Votings int,
                 Duration int,
                 Genre varchar(100))
""")

for _, row in df.iterrows():
    sql = "INSERT INTO imdb_movies_list (Title, Ratings, Votings, Duration, Genre) VALUES (%s, %s, %s, %s, %s)"
    val = (row['Title'], row['Ratings'], row['Votings'], row['Duration'], row['Genre'])
    mycursor.execute(sql, val)
    mydb.commit()

In [19]:
mycursor.execute("""CREATE TABLE IMDB.images (
    id INT AUTO_INCREMENT PRIMARY KEY,
    image_name VARCHAR(255),
    image_data LONGBLOB)""")

0

In [26]:
# Read image as binary
image_path = "Imdb_Image.png"
with open(image_path, "rb") as file:
    image_binary = file.read()

# Insert image into TiDB
sql = """INSERT INTO IMDB.images (image_name, image_data)VALUES (%s, %s)"""

mycursor.execute(sql, ("Imdb_Image.png", image_binary))
mydb.commit()

In [40]:
df.to_csv('imdb_movie_list_Cleaned.csv', index=False)